In [2]:
 import pandas as pd 
 from pybaseball import batting_stats, pitching_stats, fielding_stats
 from langchain_core.documents import Document 
 from langchain_community.embeddings import HuggingFaceEmbeddings
 from langchain_community.vectorstores import Chroma
 import shutil
 import time

In [3]:
#擷取多年份全聯盟數據
YEARS_TO_FETCH = [2022, 2023, 2024, 2025]
all_player_documents = []

print(f"擷取{YEARS_TO_FETCH}的全聯盟數據")

def format_player_as_markdown(player_row, year):

    name = player_row.get('Name',  'N/A')
    team = player_row.get('Team', 'N/A')
    if pd.isna(team) and pd.notna(player_row.get('Team_bat')):
        team = player_row.get('Team_bat')
    if pd.isna(team) and pd.notna(player_row.get('Team_pit')):
        team = player_row.get('Team_pit')

    content = f"# 球員報告: {name} ({team})\n\n"
    content += f"**賽季 (Season)**: {year}\n"
    content += f"**IDfg**: {player_row.get('IDfg')}\n"


    # batting stats
    if pd.notna(player_row.get('wOBA')):
            content += "\n## 打擊 (Batting) 概要\n"
            content += f"- Position(s): {player_row.get('POS', 'N/A')}\n"
            content += f"- PA (打席): {player_row.get('PA', 0)}\n"
            content += f"- BA (打擊率): {player_row.get('BA', 0):.3f}\n"
            content += f"- HR (全壘打): {player_row.get('HR_bat', 0)}\n"
            content += f"- wOBA (進階攻擊指數): {player_row.get('wOBA', 0):.3f}\n"
            content += f"- wRC+ (標準化得分創造): {player_row.get('wRC+', 0)} (100=平均)\n"
            content += f"- WAR (勝利貢獻值): {player_row.get('WAR_bat', 0)}\n"


    # pitching stats
    if pd.notna(player_row.get('FIP')):
            content += "\n## 投球 (Pitching) 概要\n"
            content += f"- IP (投球局數): {player_row.get('IP', 0)}\n"
            content += f"- ERA (防禦率): {player_row.get('ERA', 0)}\n"
            content += f"- FIP (投手獨立防禦率): {player_row.get('FIP', 0):.2f}\n"
            content += f"- K/9 (每九局三振): {player_row.get('K/9', 0)}\n"
            content += f"- BB/9 (每九局保送): {player_row.get('BB/9', 0)}\n"
            content += f"- WAR (勝利貢獻值): {player_row.get('WAR_pit', 0)}\n"


    # fielding stats
    if pd.notna(player_row.get('Def')):
            content += "\n## 守備 (Fielding) 概要\n"
            content += f"- Position(s): {player_row.get('Pos', 'N/A')}\n"
            content += f"- Inn (守備局數): {player_row.get('Inn', 0)}\n"
            content += f"- Def (防守價值): {player_row.get('Def', 0)}\n"

    return content


# 循環抓取每一年
for year in YEARS_TO_FETCH:
    print(f"\n ---正在處理{year}賽季---")
    try:
        h_qual = 100 if year > 2020 else 50
        p_qual = 30 if year > 2020 else 10
        f_qual = 100 if year > 2020 else 50

        hitting_df = batting_stats(year, year, qual=h_qual)
        print(f"抓到 {year} 年 {len(hitting_df)} 筆打擊數據...")

        pitching_df = pitching_stats(year, year, qual=p_qual)
        print(f"抓到 {year} 年 {len(pitching_df)} 筆投球數據...")

        fielding_df = fielding_stats(year, year, qual=f_qual)
        print(f"抓到 {year} 年 {len(fielding_df)} 筆守備數據...")


        # 合併數據
        merged_df = pd.merge(
            hitting_df, pitching_df, 
            on=['IDfg', 'Name', 'Team'], how='outer', suffixes=('_bat', '_pit')
        )
        fielding_df = fielding_df.drop(columns=['Team'], errors='ignore')
        
        full_player_df = pd.merge(
            merged_df, fielding_df,
            on=['IDfg', 'Name'], how='outer'
        )
        
        print(f"成功合併 {year} 年數據，共 {len(full_player_df)} 筆球員資料。")


        # 轉換為documents
        year_docs = 0
        for _, row in full_player_df.iterrows():
            if pd.isna(row['IDfg']):
                continue

            player_markdown = format_player_as_markdown(row, year)


            metadata = {
                "player_id_fg": int(row['IDfg']),
                "player_name": row['Name'],
                "year": year, 
                "has_batting": pd.notna(row.get('wOBA')),
                "has_pitching": pd.notna(row.get('FIP')),
                "has_fielding": pd.notna(row.get('Def'))
            }

            doc = Document(page_content=player_markdown, metadata=metadata)
            all_player_documents.append(doc)
            year_docs += 1

        print(f"成功將 {year} 年的 {year_docs} 位球員轉換為 Document。")
        time.sleep(5)

    except Exception as e:
        print(f"抓取 {year} 年數據時發生錯誤: {e}")

print(f"\n--- 總共擷取了 {len(all_player_documents)} 份球員-年份報告 ---")

# 建立vector DB
if all_player_documents:
        print("---建立Vector DB---")


        # 初始化Embedding model (all-mpnet-base-v2)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        model_kwargs = {'device': 'cuda'} # 使用 GPU
        embeddings_model = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs
        )
        print(f"Embedding 模型 ({model_name}) 載入成功，使用裝置: {embeddings_model.client.device}")

        db_directory = 'mlb_db'

        # 建立ChromaDB
        print("正在將所有 Document 存入 ChromaDB... ")
        vectordb = Chroma.from_documents(
            documents=all_player_documents,
            embedding=embeddings_model,
            persist_directory=db_directory
        )
    
        print(f"\n功建立向量資料庫並儲存於 ./{db_directory}/")
        print(f"資料庫中目前有 {vectordb._collection.count()} 筆「球員-年份」文件。")
else:
    print("沒有文件可共存入資料庫，檢查擷取多年份全聯盟數據這一步")

擷取[2022, 2023, 2024, 2025]的全聯盟數據

 ---正在處理2022賽季---
抓到 2022 年 469 筆打擊數據...
抓到 2022 年 470 筆投球數據...
抓到 2022 年 833 筆守備數據...
成功合併 2022 年數據，共 1180 筆球員資料。
成功將 2022 年的 1180 位球員轉換為 Document。

 ---正在處理2023賽季---
抓到 2023 年 461 筆打擊數據...
抓到 2023 年 476 筆投球數據...
抓到 2023 年 804 筆守備數據...
成功合併 2023 年數據，共 1169 筆球員資料。
成功將 2023 年的 1169 位球員轉換為 Document。

 ---正在處理2024賽季---
抓到 2024 年 455 筆打擊數據...
抓到 2024 年 474 筆投球數據...
抓到 2024 年 792 筆守備數據...
成功合併 2024 年數據，共 1163 筆球員資料。
成功將 2024 年的 1163 位球員轉換為 Document。

 ---正在處理2025賽季---
抓到 2025 年 461 筆打擊數據...
抓到 2025 年 475 筆投球數據...
抓到 2025 年 794 筆守備數據...
成功合併 2025 年數據，共 1156 筆球員資料。
成功將 2025 年的 1156 位球員轉換為 Document。

--- 總共擷取了 4668 份球員-年份報告 ---
---建立Vector DB---


C:\Users\Brian\AppData\Local\Temp\ipykernel_36684\3672895958.py:125: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(
c:\Users\Brian\OneDrive\桌面\交大電信碩\IR\mlb_manager_rag\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding 模型 (sentence-transformers/all-mpnet-base-v2) 載入成功，使用裝置: cuda:0
正在將所有 Document 存入 ChromaDB... 

功建立向量資料庫並儲存於 ./mlb_db/
資料庫中目前有 9336 筆「球員-年份」文件。


In [4]:
# 載入LLM和 RAG chain
import torch
from langchain_community.chat_models import ChatOllama  
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("--- 載入 RAG 查詢鏈 ---")

model_file_path = "llama3.2:3b-instruct-q4_K_M"

try:
    print(f"正在連線 Ollama 模型: {model_file_path} ...")
    llm = ChatOllama(
        base_url="http://localhost:11434",
        model=model_file_path,
        temperature=0.1,
        num_ctx=1024,  
    )
    print(" LLM (ChatOllama) 連線成功")
except Exception as e:
    print(f" 連線 LLM 失敗: {e}")
    llm = None


# 載入 Embedding model 和 vectorDB (Retriever)
if llm:
    print("\n--- 正在載入 vectorDB (Retriever) ---")

    db_directory = 'mlb_db'
    emb_model = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {'device': 'cuda'}

    embeddings_model = HuggingFaceEmbeddings(
        model_name=emb_model,
        model_kwargs=model_kwargs
    )

    vectordb = Chroma(
        persist_directory=db_directory,
        embedding_function=embeddings_model
    )

    retriever = vectordb.as_retriever(search_kwargs={"k": 10})


# 建立 RAG prompt 和 RAG chain
if llm:
    print("\n--- 正在建立 RAG 查詢鏈 ---")

    template = """
    您是一位專業的美國職棒大聯盟 (MLB) 球隊總經理助理。
    請使用「繁體中文」回答。
    您的任務是根據以下提供的「球員報告」(Context) 來回答問題。
    
    請嚴格依據提供的資料回答，不要編造資料。
    如果資料中沒有答案，請誠實地說「我無法在提供的資料中找到相關資訊」。
    
    請分析報告中的進階數據 (如 wRC+, FIP, WAR) 來支持您的結論。
    
    [球員報告 Context]:
    {context}
    
    [總經理的問題 Question]:
    {question}
    
    [您的專業回答]:
    """

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print(" RAG 查詢鏈建立成功，準備好接受提問。")


--- 載入 RAG 查詢鏈 ---
正在連線 Ollama 模型: llama3.2:3b-instruct-q4_K_M ...
 LLM (ChatOllama) 連線成功

--- 正在載入 vectorDB (Retriever) ---


C:\Users\Brian\AppData\Local\Temp\ipykernel_36684\180229924.py:16: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(



--- 正在建立 RAG 查詢鏈 ---
 RAG 查詢鏈建立成功，準備好接受提問。


C:\Users\Brian\AppData\Local\Temp\ipykernel_36684\180229924.py:41: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


In [ ]:
# 啟動 Gradio Web UI
import gradio as gr
import time
try:
    if 'rag_chain' not in globals():
        raise NameError("'rag_chain' not defined")
except NameError:
    print("錯誤： 'rag_chain' 尚未被定義。")
else:
    print("RAG 鏈已載入。正在定義 Gradio 函數...")

    def ask_assistant(message, chat_history):
        print(f"\n[Gradio 收到問題]: {message}")
        print("(RAG 系統啟動...)")
        start_time = time.time()
        
        # 執行 RAG 鏈
        response = rag_chain.invoke(message)
        
        # 檢查 response 的類型
        if isinstance(response, bytes):
            print("--- [計畫 F]: 偵測到 bytes 輸出，正在手動 decode... ---")
            try:
                final_response = response.decode('utf-8')
            except UnicodeDecodeError:
                final_response = response.decode('latin-1', errors='ignore')
        else:
            final_response = str(response) 
        
        end_time = time.time()
        print(f"(LlamaCpp 回答完畢，耗時: {end_time - start_time:.2f} 秒)")
        
        # 3. 返回手動解碼後的字串
        return final_response 
    
    # Gradio 介面
    print("\n--- Gradio 介面 ---")
    demo = gr.ChatInterface(
        fn=ask_assistant,
        title="⚾ MLB GM Assistant",
        description="輸入您對球員數據或補強目標的問題。 (模型: Llama 3.2 3B via Ollama)",
        examples=[
            "我們球隊的打線缺乏長打火力，請幫我找出 2025 賽季 wRC+ 高於 130 的外野手。",
            "比較 Aaron Judge 和 Shohei Ohtani 在 2023-2025 年的 wRC+ 趨勢。",
            "Gerrit Cole 2025 年的 FIP 和 K/9 是多少？"
        ],
        cache_examples=False 
    )
    print("\n--- Gradio Web 伺服器 ---")
    demo.launch(share=True)